# Massey Ratings
    - Matt Robinson, Yale Undergraduate Sports Analytics Group

This notebook is one of several notebooks exploring ratings systems often used in sports. All of the notebooks can be found here(github link). 

Specifically, this notebook attempts to both explain and implement the popular Massey ratings model created by [Ken Massey](http://www.masseyratings.com/). Legend has it that Massey, now a math professor, came up with this method while still an undergraduate (which I include as a fun fact and as motivation for me to figure out my life).

## The Basic Overview ##

Massey's method is also referred to as the *Point Spread Method*. To put it simply, the goal of this method is for the difference in ratings between two teams to equal the difference in score when they play each other. Mathematically, this can be easily summarized by the equation:
$$ r_i - r_j = y_k$$
where $r_i$ and $r_j$ are the ratings of teams $i$ and $j$, respectively, and $y_k$ is the point differential for game $k$, in which $i$ and $j$ play each other.

Now, of course, this equation is just the ideal. Over the course of a few games, you will get a system of equations that look just like it. Finding ratings such that it holds true for every team and every game is unrealistic. Therefore, Massey's method uses what is called the *method of least squares* to find the best approximate ratings.

## Working Through an Example ##

In order to work through the implementation of this method and understand how it works, I'm going to use an extremely simplified example. Let's imagine the Ivy league is instead the IV league and consists of only four teams who each play each other once: Harvard, Princeton, Yale, and Columbia.

Here are the results from the 2016 IV season that I scraped from the NCAA stats website:


In [3]:
import numpy as np
import pandas as pd

In [7]:
IV_df = pd.read_csv('IV_league_2016.csv')

In [9]:
IV_df

,year,month,day,team,opponent,location,team_score,opponent_score
0,2016,10,1,Columbia,Princeton,1,13,48
1,2016,10,22,Harvard,Princeton,-1,23,20
2,2016,10,28,Columbia,Yale,1,23,31
3,2016,11,5,Columbia,Harvard,-1,21,28
4,2016,11,12,Princeton,Yale,-1,31,3
5,2016,11,19,Harvard,Yale,1,14,21


The `location` column tells us if the team in the `team` column was home or away. 
* Away = -1
* Neutral = 0
* Home = 1

Since the goal of Massey's method is to find ratings whose difference gives the score differential, we need to add a column with this data:

In [10]:
IV_df['score_diff'] = (IV_df['team_score']-IV_df['opponent_score'])

In [11]:
IV_df

,year,month,day,team,opponent,location,team_score,opponent_score,score_diff
0,2016,10,1,Columbia,Princeton,1,13,48,-35
1,2016,10,22,Harvard,Princeton,-1,23,20,3
2,2016,10,28,Columbia,Yale,1,23,31,-8
3,2016,11,5,Columbia,Harvard,-1,21,28,-7
4,2016,11,12,Princeton,Yale,-1,31,3,28
5,2016,11,19,Harvard,Yale,1,14,21,-7


### The Setup ###

Now comes time to get back to our original equation:
$$ r_i - r_j = y_k $$

In this case, we are going to try to find the ratings for all four teams of the IV league: $r_h, r_p, r_y,$ and $r_c$. To do this, we look at the six league games from the IV season and construct an equation for each game. In our case, the system of equations looks like this (using the data from the table above):

$$ 
\begin{align*}
r_c - r_p &= -35 \\
r_h - r_p &=  3 \\
r_c - r_y &= -8 \\
r_c - r_h &= -7 \\
r_p - r_y &= 28 \\
r_h - r_y &= -7 \\
\end{align*}
$$


For sake of simplicity, I am going to take every equation with a negative `score_diff` and multiply through by $-1$. Now every point differential is positive and the system of equations looks like this:
$$ 
\begin{align*}
r_p - r_c &= 35 \\
r_h - r_p &=  3 \\
r_y - r_c &= 8 \\
r_h - r_c &= 7 \\
r_p - r_y &= 28 \\
r_y - r_h &= 7 \\
\end{align*}
$$

Let's now encode this system of equations in matrix form. Generally, in a season of $m$ games among $n$ different teams, there will be $m$ equations (one for each game) and $n$ unknowns (the ratings). Thus, we can create an $m \times n$ matrix to encode the equations from the $m$ games and $n$ teams. For our example with six games and four teams, the matrix $X$ looks like this:

In [29]:
X = np.array([
        [0, 1, 0, -1],
        [1, -1, 0, 0],
        [0, 0, 1, -1],
        [1, 0, 0, -1],
        [0, 1, -1, 0],
        [-1, 0, 1, 0]
    ])
X

array([[ 0,  1,  0, -1],
       [ 1, -1,  0,  0],
       [ 0,  0,  1, -1],
       [ 1,  0,  0, -1],
       [ 0,  1, -1,  0],
       [-1,  0,  1,  0]])

where each of the columns corresponds to a team, and each row is a game. The labeled output may be more clear:

In [30]:
game_num = ['game_' + _ for _ in '123456']
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
X_df = pd.DataFrame(X, index=game_num, columns=team_names)
print(X_df.to_string())

        Harvard  Princeton  Yale  Columbia
game_1        0          1     0        -1
game_2        1         -1     0         0
game_3        0          0     1        -1
game_4        1          0     0        -1
game_5        0          1    -1         0
game_6       -1          0     1         0


For example, the first row corresponds to the game between Princeton and Columbia. The winning team, Princeton, gets a $1$ in its column while the losing team, Columbia, gets a $-1$ in its column.

Our full system of equations can be written using the linear matrix equation $Xr=y$, where $X$ is the $m \times n$ matrix we just constructed, $r$ is the $n \times 1$ vector containing the ratings of each team, and $y$ is the $m \times 1$ vector of score differences from each game.

$$
\begin{bmatrix}
  0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 \\
  0 & 0 & 1 & -1 \\
  1 & 0 & 0 & -1 \\
  0 & 1 & -1 & 0 \\
  -1 & 0 & 1 & 0
\end{bmatrix} 
\begin{bmatrix}
  r_h \\
  r_p \\
  r_y \\
  r_c
\end{bmatrix}
= 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
$$

Now, you can see that it is pretty easy to construct this matrix equation from the beginning. For each game (row), just put a $1$ in the column of the winning team, a $-1$ in the column of the losing team, and encode the absolute value of the point differential (the margin of victory) in the $y$ vector.

### Solving for the Ratings ###

In any given season, it is likely that the number of games played ($m$) will be much greater than the number of teams ($n$). This means that we have more equations than unknowns. In our IV league example, we had six equations but only four unknowns. Mathematically, this means that the matrix is *overdetermined*, and thus the system of equations is likely *inconsistent*. In simpler words, this means that there is almost certainly no solution to the system of equations.

#### Unnecsary Aside: A simple example for those unfamiliar with least squares ####
Take this really simple example of two equations and one unknown. 
$$
\begin{align*}
x &= 5 \\
x &= 9
\end{align*}
$$

Obviously, there is no $x$ that satisfies both of these equations. So we are going to need to look for the best approximate solution $\bar x$. But how do we measure which is the "best" solution?

Often, we consider the best fit to be the one that minimizes the *sum of the squared errors*. To be more mathematical, for our system of $m=2$ and $n=1$ unknowns, we seek the $\hat x$ that minimizes the following:
$$
\sum_{i=1}^m (x-\bar x)^2 = (9-\bar x)^2 + (5- \bar x)^2
$$
Now I think it is intuitive that the solution is somewhere between $5$ and $9$. The first guess is probably the average of $5$ and $9$, $\bar x = 7$ But some of you may think that it would be more advantageous to choose $\bar x = 6$ or  $\bar x = 8$ since they are closer to one of the numbers, or perhaps even $\bar x = 5$ or  $\bar x = 9$ since it makes one of the eqations exactly true. Well, it turns out that the average $\bar x = 7$ is the best choice because it minimizes $\sum_{i=1}^m (x-\bar x)^2$ if you do the math out. (This is no coincidence; the *mean* is precisely that estimator that minimizes the sum of squared errors)  

So this is exactly what we are trying to with our matrix equation: find those ratings that give the best approximate answer to our system of equations, as determined by the least squares criterion.

#### The least squares solution to our matrix equation #### 

So in the case of our matrix equation $Xr=y$ that we cannot find a solution for, we seek the least squares solution $\hat r$ such that the error $E = \parallel{Xr - y}\parallel^2$ is minimized.  

To find this $\hat r$ we focus on solving what are called the "normal equations," $X^TX\hat r = X^T y$. 

#### Unnecessary Aside: Deriving the Normal Equations ####

In the machine learning courses I have seen, the normal equations are often derived using calculus. However, I find matrix calculus a bit narly, so instead I am going to use some geometry to derive them. If you want to see the calculus derivation, please see a linear algebra/machine learning textbook.

When we are trying to solve $Xr=y$, a solution exists if and only if a linear combination of the columns of $X$ can equal the $y$ vector. That is to say (in slightly more mathematical jargon), a solution exists if $y$ lies in the column space of $X$, which I'll call $C(X)$. Well since we have an overdetermined system, $y$ is almost definitely outside of $C(X)$. So what do we do? The answer is that we give up on solving exactly for $y$. Instead, let's look for $X\hat r$, the vector in $C(X)$ that is as close as possible to $y$. 

But what does close mean? It means that the error vector $e = (y - X \hat r)$ is perpendicular to the vector $X \hat r$, which is in $C(X)$. (If the reason for this is not clear to you, this [medium post](https://medium.com/@andrew.chamberlain/the-linear-algebra-view-of-least-squares-regression-f67044b7f39b) has a rather good illustration)

And when two vectors are perpendicular, we know that their dot product must be $0$. So let's do it out and solve for the approximate ratings vector $\hat r$:

$$
\begin{align*}
(X \hat r)^T (y - X \hat r) &= 0 \\
\hat r^T X^T y -\hat r^T X^T X \hat r &= 0 \\
\hat r^T X^T X \hat r &= \hat r^T X^T y \\
(\hat r^T)^{-1} \hat r^T X^T X \hat r &= (\hat r^T)^{-1} \hat r^T X^T y \\
X^T X \hat r &= X^T y \\ 
\end{align*}
$$
And, if $X^T X$ is invertible:
$$
\hat r = (X^T X)^{-1} X^T y 
$$

If anyone cares, the matrix $(X^T X)^{-1} X^T$ is often referred to as the *Moore-Penrose Pseudoinverse*.

#### Back to the example

Anyways, let's do the math out for our IV league example and attempt to solve the normal equations $X^TX\hat r = X^T y$:

First examine what the matrix product $X^TX$ looks like:

In [35]:
print("X looks like:")
print(X_df.to_string() + '\n')
print("X^T looks like:")
print(X_df.T.to_string())

X looks like:
        Harvard  Princeton  Yale  Columbia
game_1        0          1     0        -1
game_2        1         -1     0         0
game_3        0          0     1        -1
game_4        1          0     0        -1
game_5        0          1    -1         0
game_6       -1          0     1         0

X^T looks like:
           game_1  game_2  game_3  game_4  game_5  game_6
Harvard         0       1       0       1       0      -1
Princeton       1      -1       0       0       1       0
Yale            0       0       1       0      -1       1
Columbia       -1       0      -1      -1       0       0


Let's call the matrix product $M$ for short

In [39]:
M = (X.T).dot(X)
M

array([[ 3, -1, -1, -1],
       [-1,  3, -1, -1],
       [-1, -1,  3, -1],
       [-1, -1, -1,  3]])

Showing the labels that each row and column correspond to:

In [40]:
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
M_df = pd.DataFrame(M, index=team_names, columns=team_names)
print(M_df.to_string())

           Harvard  Princeton  Yale  Columbia
Harvard          3         -1    -1        -1
Princeton       -1          3    -1        -1
Yale            -1         -1     3        -1
Columbia        -1         -1    -1         3


It may also help to see the whole multiplication written out:

$$
M = X^TX =
\begin{bmatrix}
  0 & 1 & 0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 & 1 & 0 \\
  0 & 0 & 1 & 0 & -1 & 1 \\
  -1 & 0 & -1 & -1 & 0 & 0
\end{bmatrix} 
\begin{bmatrix}
  0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 \\
  0 & 0 & 1 & -1 \\
  1 & 0 & 0 & -1 \\
  0 & 1 & -1 & 0 \\
  -1 & 0 & 1 & 0
\end{bmatrix} 
=
\begin{bmatrix}
  3 & -1 & -1 & -1 \\
  -1 & 3 & -1 & -1 \\
  -1 & -1 & 3 & -1 \\
  -1 & -1 & -1 & 3
\end{bmatrix}  
$$

$M$ is an $n \times n$ matrix. In general, the diagonal entries $M_{ii}$ correspond to the total number of games played by team $i$. The other entries $M_{ij}$ for $i \neq j$ are simply equal to the negation of how many times team $i$ played team $j$.

Now let's move on to the right side of the equation, $X^T y$. For sale of simplicity, we will call this vector $p$.

In [43]:
yT = np.array([
        [35, 3, 8, 7, 28, 7]
    ])
y = yT.T
y

array([[35],
       [ 3],
       [ 8],
       [ 7],
       [28],
       [ 7]])

In [49]:
p = (X.T).dot(y)
p

array([[  3],
       [ 60],
       [-13],
       [-50]])

In [68]:
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
p_df = pd.DataFrame(p, index=team_names,columns=[''])
print(p_df.to_string())

             
Harvard     3
Princeton  60
Yale      -13
Columbia    0


Some of you may notice something special about this vector $p$. I'll write the multiplication out in full so that it may be more clear to you.

$$
p = X^Ty =
\begin{bmatrix}
  0 & 1 & 0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 & 1 & 0 \\
  0 & 0 & 1 & 0 & -1 & 1 \\
  -1 & 0 & -1 & -1 & 0 & 0
\end{bmatrix} 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
=
\begin{bmatrix}
  3 \\
  60 \\
  -13 \\
  -50 
\end{bmatrix} 
$$

The vector $p$ encodes the total point differential, summed across every game for a given team. Let's just examine the first row of $p$, which encodes Harvard's total point differential. 

Harvard played three games in the IV league. In their first game they beat Princeton with a point differential of +3. In their second game, they beat Columbia with a point differential of +7. In their last game, they lost to Yale by a point differential of -7. Alltogether, their total point is 7 + 3 -7 = 3.

If you do the matrix multiplaction for the first row, the same exact thing is happening. You add all the point differentials from Harvard's win and subtract the point differentials from the loss. Thus, $p$ simply encodes the cumulative point differentials for each team.

Now, it is also pretty easy to see that both $M$ and $p$ can be constructed pretty easily from the beginning without having to first construct $X$ and $y$. This avoids some unnecessary computation. Also note that $M$ is $n \times n$ and is thus ususally much smaller than the $m \times n$ matrix $X$.

#### Finally Solving the Normal Equations.

Now that we have seen how to construct $M$ and $p$, let's solve for $\hat r$ using the normal equations $M \hat r = p$.

$$
\begin{bmatrix}
  3 & -1 & -1 & -1 \\
  -1 & 3 & -1 & -1 \\
  -1 & -1 & 3 & -1 \\
  -1 & -1 & -1 & 3
\end{bmatrix}
\begin{bmatrix}
  r_h \\
  r_p \\
  r_y \\
  r_c
\end{bmatrix}
=
\begin{bmatrix}
  3 \\
  60 \\
  -13 \\
  -50 
\end{bmatrix} 
$$

(Note: I probably should put hats over the individual ratings in $\hat r$, but by now I think it is clear that these ratings will be an approximation.)

But before we solve for the ratings, it is worth also examining the columns of $M$ to spot **one minor issue**. Some of you may have seen immedietly this interesting property of the columns of $M$:

$$
\begin{bmatrix}
   3 \\
  -1 \\
  -1 \\
  -1 
\end{bmatrix}  
+ 
\begin{bmatrix}
  -1 \\
  3 \\
  -1 \\
  -1 
\end{bmatrix} 
+
\begin{bmatrix}
  -1 \\
  -1 \\
  3 \\
  -1 
\end{bmatrix} 
+ 
\begin{bmatrix}
  -1 \\
  -1 \\
  -1 \\
  3 
\end{bmatrix} 
= 
\begin{bmatrix}
  0 \\
  0 \\
  0 \\
  0 
\end{bmatrix} 
$$

Some of you will also remember that a set of vectors is *linearly dependent* if one of the vectors can be written as a linear combination of the others vectors. Well let me just move one of the vectors over to the other side and multiply the equation by $-1$:

$$
-1*
\begin{bmatrix}
   3 \\
  -1 \\
  -1 \\
  -1 
\end{bmatrix}  
+ 
-1*
\begin{bmatrix}
  -1 \\
  3 \\
  -1 \\
  -1 
\end{bmatrix} 
+
-1*
\begin{bmatrix}
  -1 \\
  -1 \\
  3 \\
  -1 
\end{bmatrix} 
= 
\begin{bmatrix}
  -1 \\
  -1 \\
  -1 \\
  3 
\end{bmatrix} 
$$

It's now pretty obvious that this set of vectors is linearly dependent. You may similarly notice that the same is true of the rows of $M$. These properties also hold true in general when we consider the matrix $M$ over an arbitrary set of games and teams.

The linear dependence of the columns of $M$ has some interesting properties that we need to consider. The first thing you may remember from a linear algebra class is that linearly dependent columns is one of the many, many equivalent ways to say that a matrix is not invertible. Also, we observe that $rank(M) < n$, and thus the solution for $\hat r$ is not unique.

Massey came up with a **clever fix** for this. He simply replaces the last row of $M$ with a row of all ones. and sets the last entry of $p$ to be $0$. This new system is now denoted $\bar M \hat r = \bar p$ and looks like this:

$$
\begin{bmatrix}
  3 & -1 & -1 & -1 \\
  -1 & 3 & -1 & -1 \\
  -1 & -1 & 3 & -1 \\
  1 & 1 & 1 & 1
\end{bmatrix}
\begin{bmatrix}
  r_h \\
  r_p \\
  r_y \\
  r_c
\end{bmatrix}
=
\begin{bmatrix}
  3 \\
  60 \\
  -13 \\
  0 
\end{bmatrix} 
$$

$\bar M$ is now full rank and there is a unique solution for $\hat r$. There is one other nice property of this new setup which can be seen by doing the matrix multiplication for the last row. 
$$
1*r_h + 1*r_p + 1*r_y + 1*r_c = 0
$$
That is, all of the rankings must now sum to zero.

Let's now finally get a solution using numpy for the ratings:

In [58]:
M_bar = M
M_bar[-1,:] = np.ones(M.shape[0])
M_bar

array([[ 3, -1, -1, -1],
       [-1,  3, -1, -1],
       [-1, -1,  3, -1],
       [ 1,  1,  1,  1]])

In [59]:
p_bar = p
p_bar[-1] = 0
p_bar

array([[  3],
       [ 60],
       [-13],
       [  0]])

In [62]:
r_hat = np.linalg.inv(M_bar).dot(p_bar)
r_hat

array([[  0.75],
       [ 15.  ],
       [ -3.25],
       [-12.5 ]])

In [67]:
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
r_df = pd.DataFrame(r_hat, index=team_names,columns=[''])
print(r_df.to_string())

                
Harvard     0.75
Princeton  15.00
Yale       -3.25
Columbia  -12.50


## The Connection Between Massey's Method and Linear Regression

I am now going to try to motivate Massey's Method a bit differently. At YUSAG, we use a linear regression model for our rankings. Linear regression is a classical statistical method that also relies on the method of least squares in determining the proper paramemters. Below I will attempt to show that Massey's Method is equivalent to linear regression, which offers a nice, alternative interpretation for why it works. 

### Basics of Linear Regression

Here, I am going to provide a very, very brief overview of how linear regression works. I suggest you consult any of the countless excellent resources on the topic if you wish to dive deeper.

#### The Form of Multiple Linear Regression

In linear regression, we simply attempt to predict a scalar variable $\hat y$ as the weighted sum of a bunch of input features (explanatory variables) and a bias term (the interecept). The general form is:

$$
\hat y = w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_n x_n
$$
where:
* $\hat y$ is the predicted response variable
* $x_i$ is the i'th feature value
* $w_j$ is the j'th model parameter
    * $w_0$ is the bias term
    * $w_1, w_2,...,w_n$ are the feature weights
* $n$ is the number of features

So how do we figure out the values of the model coefficients $w_0,...,w_n$? The answer is that we learn these parameters when we train the linear model on the data. In fact, for linear regression, the fit is determined using the least squares criterion. That is, we seek the parameters that minimize the sum of squared errors. Once the model is trained, and the best parameters are learned, we can use the model for prediction.

It turns out there is an even nicer way to write the general form of linear regression using vector notation:

$$
\hat y = w^Tx
$$

where $w$ is the parameter vector containing the bias term $w_0$ and the feature weights $w_1,...,w_n$. The vector $x$ is the feature vector containing $x_0$, which is always equal to $1$, and the feature values $x_1,...,x_n$.

And, if there is a whole training set of data, we construct a matrix $X$ whose rows are feature vectors $x$. We also construct a vectory $y$ which containes the target values for each data point of feature values. In matrix form:

$$
Xw = y
$$

#### The Data ####

For this linear regression problem, I am going to load the data from the 2016 IV season in a specialized format.

In [73]:
lin_reg_df = pd.read_csv('IV_linear_regression_data.csv')
lin_reg_df

,year,month,day,Harvard,Princeton,Yale,Columbia,location,MOV
0,2016,10,1,0,1,0,-1,-1,35
1,2016,10,22,1,-1,0,0,-1,3
2,2016,10,28,0,0,1,-1,-1,8
3,2016,11,5,1,0,0,-1,1,7
4,2016,11,12,0,1,-1,0,-1,28
5,2016,11,19,-1,0,1,0,-1,7


Some explanation of this data is in order. Each row represents a game. Within each row, the feature value for the winning team is $1$ and the value for the losing team is $-1$. All the other teams receive a value of $0$. The `location` column corresponds to the location of the winning team, with $1$ being home, $-1$ being away, and $0$ being neutral. Lastly, the `MOV` column is simply the margin of victory.

For example, in the game on 10/01/2016, Princeton beat Columbia at Columbia by 35 points.

#### Seeting Up Our Model ####

Broadly, the goal of our linear regression model is to explain the margin of victory (MOV) based on the strength of the winning team and the strength of the losing team. (I am going to exclude location for now)

Let's first make our training data:

In [75]:
X = lin_reg_df.drop(['year','month','day','location','MOV'], axis=1)
X

,Harvard,Princeton,Yale,Columbia
0,0,1,0,-1
1,1,-1,0,0
2,0,0,1,-1
3,1,0,0,-1
4,0,1,-1,0
5,-1,0,1,0


In [77]:
y = lin_reg_df['MOV']
y

0    35
1     3
2     8
3     7
4    28
5     7
Name: MOV, dtype: int64

Now let's set up the matrix equation $Xw = y$ that we are trying to solve. Remember that we must add $x_0$ and $w_0$. I am also going to set up change the weight subscripts from $1,...,n$ to the name of the schools that the weights refer to. 

$$
\begin{bmatrix}
  1 & 0 & 1 & 0 & -1 \\
  1 & 1 & -1 & 0 & 0 \\
  1 & 0 & 0 & 1 & -1 \\
  1 & 1 & 0 & 0 & -1 \\
  1 & 0 & 1 & -1 & 0 \\
  1 & -1 & 0 & 1 & 0
\end{bmatrix} 
\begin{bmatrix}
  w_0 \\
  w_h \\
  w_p \\
  w_y \\
  w_c 
\end{bmatrix}
= 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
$$

Mathematically, our linear model is a function of a bias term and four input features: `Harvard`, `Princeton`, `Yale`, and `Columbia`.

$$
\tt MOV = w_0 + (w_h*{Harvard}) + (w_p*{Princeton}) + (w_y*\tt{Yale}) + (w_c*\tt{Columbia})
$$

Let's examine what we get from multiplying the first row of $X$ by $w$:

$$
\tt 35 = (1*w_0) + (w_h*0) + (w_p*1) + (w_y*0) + (w_c*{-1}) \\
\tt 35 = w_0 + w_p - w_c
$$

Now, for the sake of argument, let's imagine that we set the bias term $w_0$ to be always $0$. Now the equation from the first row looks like this: 

$$
\tt 35 = w_p - w_c
$$

This looks a lot like the ideal equation from Massey's method, just with the model coefficients $w_p$ and $w_c$ replacing the ratings $r_p$ and $w_c$. That is, the difference in the model parameters for Princeton and Columbia should equal the difference in score when they play each other. If we do this for the other rows, the equations will all look similar too.

Still insisting that the bias term $w_0=0$, our full linear model looks like this: 

$$
\tt MOV = (w_h*{Harvard}) + (w_p*{Princeton}) + (w_y*\tt{Yale}) + (w_c*\tt{Columbia})
$$

And now that we no longer care about finding $w_0$, our matrix equation $Xw = y$ looks like this:

$$
\begin{bmatrix}
  0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 \\
  0 & 0 & 1 & -1 \\
  1 & 0 & 0 & -1 \\
  0 & 1 & -1 & 0 \\
  -1 & 0 & 1 & 0
\end{bmatrix} 
\begin{bmatrix}
  w_h \\
  w_p \\
  w_y \\
  w_c
\end{bmatrix}
= 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
$$

Look familiar? That's because this is exactly the matrix equation we got using Massey's method, just with the ratings now called weights. So this shows that Massey's method is just linear regression with the bias term set to $0$. Again, we can just solve using the method of least squares and get the desired weights, which can be interpreted as ratings.

#### Finding the Linear Regression Parameters ####

I am going to go back to the original linear regression equation: 

$$
\begin{bmatrix}
  1 & 0 & 1 & 0 & -1 \\
  1 & 1 & -1 & 0 & 0 \\
  1 & 0 & 0 & 1 & -1 \\
  1 & 1 & 0 & 0 & -1 \\
  1 & 0 & 1 & -1 & 0 \\
  1 & -1 & 0 & 1 & 0
\end{bmatrix} 
\begin{bmatrix}
  w_0 \\
  w_h \\
  w_p \\
  w_y \\
  w_c 
\end{bmatrix}
= 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
$$

This one is again an overdetermined system and can be solved using the least squares method and the normal equations. I am going to use this version, because it is truly linear regression and will be easier to train using linear regression packages.

In [89]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression(fit_intercept=False)
linreg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [94]:
# print the coefficients
print(linreg.intercept_)
print(linreg.coef_)

12.6666666667
[ -2.41666667  11.83333333  -6.41666667  -3.        ]


In [92]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression(fit_intercept=True)
linreg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [95]:
# print the coefficients
print(linreg.intercept_)
print(linreg.coef_)

12.6666666667
[ -2.41666667  11.83333333  -6.41666667  -3.        ]


In [83]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge()
ridge_reg.fit(X, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [84]:
# print the coefficients
print(ridge_reg.intercept_)
print(ridge_reg.coef_)

13.3333333333
[-2.06666667  9.33333333 -5.26666667 -2.        ]
